In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
from GE2E import get_emb
import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import logging

numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)
numba_logger = logging.getLogger('matplotlib')
numba_logger.setLevel(logging.WARNING)
import commons
import utils
from data_utils import TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [2]:
hps = utils.get_hparams_from_file("configs/cjks_base.json")

In [27]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint("ckpts/G_323000.pth", net_g, None)

INFO:root:Loaded checkpoint 'ckpts/G_323000.pth' (iteration 578)


In [28]:
stn_tst = get_text("[JA]こんにちは[JA]", hps)
g = torch.FloatTensor(get_emb("abc.wav")).unsqueeze(0)
with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    # print(stn_tst)
    audio = net_g.infer(x_tst, x_tst_lengths,sid=g, noise_scale=.3, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))